In [1]:
import sys; sys.path.insert(0, "../")
import pandas as pd
import numpy as np

from clean.new_orleans_csd_pprr import clean_2014, clean_2009
from lib.explore import show_unique
from lib.match import (
    ThresholdClassifier, ColumnsIndex, StringSimilarity, JaroWinklerSimilarity,
    DateSimilarity, match_records, print_match_result
)
from lib.date import combine_date_columns
from lib.uid import gen_uid

In [2]:
df14 = clean_2014()
df14

,department_desc,last_name,first_name,rank_code,rank_desc,annual_salary,birth_year,birth_month,birth_day,hire_year,...,hire_day,term_year,term_month,term_day,pay_prog_start_year,pay_prog_start_month,pay_prog_start_day,mid,data_production_year,agency
0,pol managemnt services bureau,moore,charmaine,C0025,clerical specialist,11235.83,1901,1,1,1996,...,8,1996,8,28,1996,8,28,d9b2bb8829f4b2e4fd97290ea93ee56e,2014,New Orleans CSD
1,pol managemnt services bureau,coxen,shanna,C5201,laboratory technician 1,14951.72,1972,10,5,1996,...,22,1997,1,1,1997,1,1,693a1967b92e800cfca8b5f800030d47,2014,New Orleans CSD
2,pol special investigation div,cavaliere sr,morris,C7134,police officer 4,29243.06,1947,7,21,1970,...,21,1997,1,4,1997,1,4,0e1583f54d720cd899d9ce17c5d12869,2014,New Orleans CSD
3,pol first district,fonseca,norbert,C7112,police lieutenant,39893.17,1943,8,31,1967,...,21,1997,1,4,1997,1,4,fff2935b97640caa5e15508cf21a871c,2014,New Orleans CSD
4,pol fifth district,hartman,steven,C7112,police lieutenant,33524.95,1964,3,11,1985,...,10,1997,1,4,1997,1,4,7b5f6f0e0cd3b2cfe3718d64a535ac43,2014,New Orleans CSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,pol managemnt services bureau,johnson,shawn,C7105,police aide,28882.03,1990,6,23,2014,...,17,2014,12,10,2014,8,17,3e28e9f4250a2e3ae76d6f36345c6153,2014,New Orleans CSD
3682,pol sixth district,linehan,henry,C7110,police officer 1,37490.69,1982,8,18,2009,...,27,2014,12,11,2011,3,13,7693cd75ee286e1b018fe5df832f79dc,2014,New Orleans CSD
3683,pol first district,finneman,david,C7111,police sergeant,50513.12,1960,12,20,2001,...,16,2014,12,13,2013,1,30,a4180ae4b9be2d1a19f2f68088cc6254,2014,New Orleans CSD
3684,pol managemnt services bureau,badr,mohamed,C7105,police aide,28882.03,1980,10,6,2014,...,31,2014,12,20,2014,8,31,eb56dc40d6c443298c510e1925494a56,2014,New Orleans CSD


In [3]:
df09 = clean_2009()
df09

,department_desc,last_name,first_name,rank_code,rank_desc,annual_salary,term_year,term_month,term_day,pay_prog_start_year,pay_prog_start_month,pay_prog_start_day,mid,data_production_year,agency
0,fiscal management,abbott,christopher,C7134,police officer 4,46885.0056,,,,1992,2,9,a83da7a1db81c75a740615457b6c743c,2009,New Orleans CSD
1,traffic,abbott,david,C7134,police officer 4,48064.4565,,,,1987,11,1,3dcc3b0764c84a77471669018ec797b0,2009,New Orleans CSD
2,police recruits,abram,shelton,C7109,police recruit academy,34797.9137,,,,2009,9,6,389bde7e51c4fda4c1a3b1fb0272be42,2009,New Orleans CSD
3,fiscal management,adams,james,C7133,police officer 3,45734.4971,2009,8,30,1987,9,6,5411afa95624ca1f17b558ca73412d85,2009,New Orleans CSD
4,public integrity,adams,bruce,U0018,assistant supt of police,106440.2966,,,,1975,10,16,2cfb603b68dc1acc98c43543ad027473,2009,New Orleans CSD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,police recruits,yount,jeffrey,C7109,police recruit academy,34797.9137,,,,,,,1021900d49125740b1e615266d27d6ab,2009,New Orleans CSD
1977,technicial services burea,zaldivar,sara,C7101,police complaint operator,25508.0588,,,,,,,b0f1f728e28fb1b3cfbb99a027cd77b2,2009,New Orleans CSD
1978,sixth district,zapico,daniel,C7110,police officer 1,36570.7051,,,,,,,747ca1d58bc282627e88a0583cf6c700,2009,New Orleans CSD
1979,third district,zoller,ron,C7134,police officer 4,44612.2209,,,,,,,1f52e3b5c21fdd781dddd6f32d242e9b,2009,New Orleans CSD


In [4]:
dfa = df09[["first_name", "last_name", "rank_code", "mid"]]
dfa.loc[:, "hire_date"] = combine_date_columns(df09, "pay_prog_start_year", "pay_prog_start_month", "pay_prog_start_day")
dfa = dfa.drop_duplicates(["mid"]).set_index("mid", drop=True)
dfb = df14[["first_name", "last_name", "rank_code", "mid"]]
dfb.loc[:, "hire_date"] = combine_date_columns(df14, "hire_year", "hire_month", "hire_day")
dfb = dfb.drop_duplicates(["mid"]).set_index("mid", drop=True)
matches, potential_matches, non_matches = match_records(
    ColumnsIndex(["first_name"]),
    ThresholdClassifier(
        fields={
            "last_name": JaroWinklerSimilarity(),
            "rank_code": StringSimilarity(),
            "hire_date": DateSimilarity()
        },
        thresholds=[0.8, 0.7]
    ),
    dfa,
    dfb
)

In [5]:
# len(matches)
# len(potential_matches)
print_match_result(dfa, dfb, matches, potential_matches, [])


*** MATCHES ***

                                 first_name last_name rank_code  hire_date
66e75a8e37abef81e776142e7371931d    chetley    ballex     C7134 1980-06-22
66e75a8e37abef81e776142e7371931d    chetley    ballex     C7134 1980-06-22

                                 first_name last_name rank_code  hire_date
c6bcb10359ce430b55f8ac173e7d153c    krister     vilen     C7134 1997-12-14
c6bcb10359ce430b55f8ac173e7d153c    krister     vilen     C7134 1997-12-14

                                 first_name last_name rank_code  hire_date
69229d78b128f4626069191482e6c457       eris  williams     C7141 2009-07-12
69229d78b128f4626069191482e6c457       eris  williams     C7141 2009-07-12

                                 first_name last_name rank_code  hire_date
56a31c379d4aeef0be619f87cf14036d   tyneshia    booker     C7109 2008-11-23
56a31c379d4aeef0be619f87cf14036d   tyneshia    booker     C7109 2008-11-23

                                 first_name     last_name rank_code  hire_date

In [6]:
len(matches)

325